### This code used MR prostate dataset as example.

Original directory: /cmvm.datastore.ed.ac.uk/cmvm_datastore/smgphs/groups/OncologyPhysics/clinical_data/prostate/MR_Prostate_Project/MR_Prostate_New_Anon_Data/Original MR T2 Data/001

# 1. Import packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pydicom as dicom
import dicom_contour.contour as dcm
import cv2
import functions_v2 


# 2. Read in RT files

In [ ]:
# directory of MRI file
path = '001'

## 2.1 Read in image files (DICOM format)

In [ ]:
ordered_slices = functions_v2.order_slice(path)
print(ordered_slices) # ordered slice filenames with its index and z position

ordered_slices[:, 0] --> the file names of the ordered slices  
ordered_slices[:, 1] --> the indices of the ordered slices  
ordered_slices[:, 2] --> the z positions of ordered slices

In [ ]:
# generate the ordered image set
images = [dicom.read_file(path + '/' + f).pixel_array for f in ordered_slices[:, 0]] # read in filenames iteratively
plt.imshow(images[1], cmap = 'gray') # showing the second slice 'MR010019.dcm'
plt.title(f'{ordered_slices[1][0]}')

## 2.2 Read in structure file

In [ ]:
# read in structure file
structure_file = functions_v2.get_contour_file(path)
structure_file

In [ ]:
# sanity check
contour_data = dicom.read_file(structure_file)
contour_sequence = dcm.get_roi_names(contour_data)
contour_sequence

In [ ]:
# specify the contour
contour_name = 'bladder'
# locate the contour in the contour sequence
contour_no = functions_v2.get_contour_sequence(structure_file, contour_name)
contour_no # Python counts from 0

The get_contour_dict function includes converting the contour matrics from cartesian coordinates to pixel coordinates. 


In [ ]:
# find out the slices that contains the specified contour and match them together.
ima_contour_pairs = functions_v2.get_contour_dict(structure_file, path, contour_no)
ima_contour_pairs

In [ ]:
slices_with_contours = functions_v2.get_slices_with_contours(structure_file, path, contour_no, ordered_slices)
slices_with_contours

In [ ]:
slices_with_contours[1] # the second slice that contain the contour

In the next cell, you can see that we used slices_with_contour[x] to index.

slices_with_contours[1][0] --> 'MR010019.dcm', the filename of the second slice that contained the contour  
slices_with_contorus[1][1] --> 1, the index of the second slice that contained the contour in ordered_slices

In [ ]:
# visulisation for sanity check
plt.figure(figsize = (10,10))
plt.subplot(121)
plt.imshow(ima_contour_pairs[slices_with_contours[1][0]][0], cmap = 'gray') # show the second slice that contain the contour
plt.title(f'{slices_with_contours[1][0]} image')
plt.subplot(122)
plt.imshow(ima_contour_pairs[slices_with_contours[1][0]][1], cmap = 'gray')
plt.title(f'{slices_with_contours[1][0]} contour')

# 3 Define region of interest and feature calculation

## 3.1 For hollw organ

In [ ]:
# generate 2-pixel-wide bladder wall
width = 2
images, contours, masks, expanded_contours = functions_v2.get_image_expandedContour_files(path, contour_no, width)

# visualisation for sanity check
plt.figure(figsize = (10, 8))
plt.subplot(221)
plt.imshow(images[slices_with_contours[1][1]], cmap = 'gray') # show the second slice that contain the contour
plt.title(f'{slices_with_contours[1][0]} MR image')
plt.subplot(222)
plt.imshow(contours[slices_with_contours[1][1]], cmap = 'gray')
plt.title(f'{slices_with_contours[1][0]} bladder contour')
plt.subplot(223)
plt.imshow(masks[slices_with_contours[1][1]], cmap = 'gray')
plt.title(f'{slices_with_contours[1][0]} bladder mask')
plt.subplot(224)
plt.imshow(expanded_contours[slices_with_contours[1][1]], cmap = 'gray')
plt.title(f'{slices_with_contours[1][0]} exapnded 2-pixel-wide bladder mask')

2-pixel-wide bladder wall is too small to extract features, generate 8-pixel-wide bladder wall by expanding the 2-pixel-wide bladder wall inwards and outwards with 3 pixels respectively so that we can extract 8x8 subimages.

The following cell is just for sanity check. The function **expand_two_sides** is wrapped in function **generate_features**. You can use **generate_feautres** directly.


In [ ]:
mask = masks[slices_with_contours[1][1]] # the mask of the second slice with contours  
outward_expansion_size = 3
target_size = 8
# plase read the explanation under function expand_two_sides in functions_v2 for usage
expanded_bladder_wall = functions_v2.expand_two_sides(mask, outward_expansion_size, target_size)

# visualisation for sanity check
# visualisation for sanity check
plt.figure(figsize = (15, 8))
plt.subplot(131)
plt.imshow(contours[slices_with_contours[1][1]], cmap = 'gray') # show the second slice that contain the contour
plt.title(f'{slices_with_contours[1][0]} bladder contour')
plt.subplot(132)
plt.imshow(expanded_contours[slices_with_contours[1][1]], cmap = 'gray')
plt.title(f'{slices_with_contours[1][0]} 2-pixel-wide bladder wall')
plt.subplot(133)
plt.imshow(expanded_bladder_wall, cmap = 'gray')
plt.title(f'{slices_with_contours[1][0]} 8-pixel-wide bladder wall')

We calculate radiomic features based on IBSI (the image biomarker standardisation initiatives)

images quantisation (16 bins) is included in function  *generate_feautres*.

https://ibsi.readthedocs.io/en/latest/

In [ ]:
# radiomic features for all subimages on MR010019.dcm (the second slice that contain the contour)
features = functions_v2.generate_features(images[slices_with_contours[1][1]], 
                                          masks[slices_with_contours[1][1]], 
                                          ROI_size = 8,
                                          ifHollow = True, 
                                          outward_expansion_size = 3)
features

In [ ]:
# show the results of the first subimage on MR010019.dcm
# you can adjust the code to store the results of all subimages on MR010019.dcm

FOS = ['FOS' + str(i+1) for i in range(23)]
GLCM = ['GLCM' + str(i+1) for i in range(25)]
GLRLM = ['GLRLM' + str(i+1) for i in range(16)]
GLSZM = ['GLSZM' + str(i+1) for i in range(16)]
GLDZM = ['GLDZM' + str(i+1) for i in range(16)]
NGTDM = ['NGTDM' + str(i+1) for i in range(5)]
NGLDM = ['NGLDM' + str(i+1) for i in range(17)]

feature_names = [FOS + GLCM + GLRLM + GLSZM + GLDZM + NGTDM + NGLDM]

feature_values= [features[0][0] + features[0][1] + features[0][2] + features[0][3] + features[0][4] + features[0][5] + features[0][6]]

results = dict(zip(feature_names[0], feature_values[0]))
results

## 3.1 For solid organ

assume that the bladder is a solid organ, and we caluclate radiomic features from the whole bladder mask, rather than an expanded bladder wall.

All you need to do is set **ifHollow = False**.

It would take some time to run the next cell since a solid mask is much larger than an expanded ring-like mask, which would generate much more subimages.

In [ ]:
# radiomic features for all subimages on MR010019.dcm (the second slice that contain the contour)
features = functions_v2.generate_features(images[slices_with_contours[1][1]], 
                                          masks[slices_with_contours[1][1]], 
                                          ROI_size = 8,
                                          ifHollow = False)
features